In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [4]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA


import pandas as pd
import numpy as np
import os
import re
import random
from matplotlib import pyplot as plt

import fns_models as fns


from subprocess import check_output
print(check_output(['ls', 'data']).decode('utf-8'))

% matplotlib inline

ImportError: No module named xgboost

In [7]:
# Pick top 3 authors
def get_top_author(num_author=3):
    train = pd.read_csv("data/train_hist_author_knn.csv")
    test = pd.read_csv("data/test_hist_author_knn.csv")

    print "[INFO] The size of train histogram for Random Forest" + str(train.shape)
    print "[INFO] The size of test histogram for Random Forest" + str(test.shape)

     
    train.iloc[:,2:-2] = train.iloc[:, 2:-2]\
        .apply(lambda x: x.astype(np.float) / (x.sum()/3), axis = 1, raw = True)
    test.iloc[:,2:-2] = test.iloc[:, 2:-2]\
        .apply(lambda x: x.astype(np.float) / (x.sum()/3), axis = 1, raw = True)
        
    author_index = train.author_id.value_counts().index[:num_author]
    train = train.loc[train['author_id'].isin(author_index)]
    test = test.loc[test['author_id'].isin(author_index)]
    
    
    print train.author_id.value_counts().head(10)
    print "[trian above] " + '=' * 50 + "[test below]"
    print test.author_id.value_counts().head(10)
    

    train_labels = train.author_id
    print train_labels.shape
    print  train.shape
    train = train.drop(['author_id', 'painting_id'], axis=1)
    test_labels = test.author_id
    test = test.drop(['author_id', 'painting_id'], axis=1)
   
    return train, train_labels, test, test_labels



[INFO] The size of train histogram for Random Forest(49890, 34)
[INFO] The size of test histogram for Random Forest(12473, 34)


In [ ]:
train, train_labels, test, test_labels = fns.get_top_author(3)

================================================================================================================

# Bayesian Optimization

[bayesian-optimization](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

================================================================================================================

In [ ]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)


    cv_result = xgb.cv(params, xgtrain, num_boost_round=num_rounds, nfold=5,
             seed=random_state,
             callbacks=[xgb.callback.early_stop(50)])

    return -cv_result['test-mae-mean'].values[-1]


def xgb_pca_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)


    cv_result = xgb.cv(params, pca_transformed, num_boost_round=num_rounds, nfold=5,
             seed=random_state,
             callbacks=[xgb.callback.early_stop(50)])

    return -cv_result['test-mae-mean'].values[-1]


In [ ]:
xgtrain, xgtrain_labels, xgtest, xgtest_labels = fns.get_top_author(3)

def xgb_bo(xgb_fn = xgb_evaluate):
    start_time = time.time()
    
    num_rounds = 3000
    random_state = 2017
    num_iter = 25
    init_points = 5
    params = {
        'eta': 0.1,
        'silent': 1,
        'eval_metric': 'mae',
        'verbose_eval': True,
        'seed': random_state
    }

    xgbBO = BayesianOptimization(xgb_fn, {'min_child_weight': (1, 20),
                                                'colsample_bytree': (0.1, 1),
                                                'max_depth': (5, 15),
                                                'subsample': (0.5, 1),
                                                'gamma': (0, 10),
                                                'alpha': (0, 10),
                                                })

    xgbBO.maximize(init_points=init_points, n_iter=num_iter)
    print('-' * 53)
    print '\n%f' % (time.time() - start_time)

    print('-' * 53)
    print('Final Results')
    print('XGboost: %f' % xgbBO.res['max']['max_val'])
    print('-' * 53)
    print('XGboost: %f' % xgbBO.res['max'])
    fns.plot_bo(xgb_fn, xgbBO)


In [ ]:
# Run BO for color histogram
xgb_bo(xgb_evaluate)

In [ ]:
# Choose the best params from BO results then predict 
best_min_child_weight = 
best_colsample_bytree = 
best_subsample = 
best_gamma = 
best_alpha = 
best_xgb = xgb(n_jobs = 4, min_child_weight=best_min_child_weight, colsample_bytree=best_colsample_bytree
               max_depth = best_max_depth, subsample = best_subsample, gamma=gamma, alpha=best_alpha)

best_xgb.fit(train, train_labels)

# use the best params to predict
xgb_true, xgb_pred = test_labels, best_xgb.predict(test)
xgb_pred

In [ ]:
test_data_df = fns.result_table(xgb_true, xgb_pred)
test_data_df

# PCA + XGB

In [5]:
## Get 15 principal components
xgb_pca = PCA(n_components=15)
xgb_pca.fit(train)
xgb_pca_transformed = xgb_pca.transform(train)
xgb_pca_transformed_test = xgb_pca.transform(test)

NameError: name 'PCA' is not defined

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.title("Scree Plot: 10 Principal Components")
plt.xlabel("Number of Components")
plt.ylabel("% of Explained Variance")
sum(pca.explained_variance_ratio_)

In [ ]:
# Run BO for pca of color histogram
xgb_bo(xgb_pca_evaluate)

In [ ]:
# Choose the best params from BO results then predict 
best_min_child_weight = 
best_colsample_bytree = 
best_subsample = 
best_gamma = 
best_alpha = 
best_xgb = xgb(n_jobs = 4, min_child_weight=best_min_child_weight, colsample_bytree=best_colsample_bytree
               max_depth = best_max_depth, subsample = best_subsample, gamma=gamma, alpha=best_alpha)

best_xgb.fit(xgb_pca_transformed, train_labels)

# use the best params to predict
xgb_true, xgb_pred = test_labels, best_xgb.predict(xgb_pca_transformed_test)
xgb_pred

In [ ]:
best_xgb.score(xgb_pca_transformed, train_labels)
print "Accuracy of best xgb model on training: %s" % str(best_xgb.score)

best_xgb.score(xgb_pca_transformed_test, test_labels)
print "Accuracy of best xgb model on testing: %s" % str(best_xgb.score)


In [ ]:

test_data_df = fns.result_table(xgb_true, xgb_pred)
test_data_df